In [4]:
# !pip install langchain
# !pip install langchain-community langchain-neo4j langchainhub neo4j
# !pip install openai langchain-openai

##### Create a Langchain application


In [ ]:
import os
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

response = llm.invoke("What is Neo4j?") #LLM.Invoke'a bir soru aktarabilir ve bir yanıt alabilirsiniz.

print(response)

Neo4j, Neo4j Inc. tarafından geliştirilen açık kaynaklı, NoSQL grafik veritabanı yönetim sistemidir. Nodes, relationships ve properties kullanarak verileri grafik benzeri bir yapıda depolamak ve yönetmek için tasarlanmıştır.

##### Prompts


In [15]:

"""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
"""
# Bu prompt template, LLM'ye context sağlayacak ve ona meyve ve sebze satan bir satıcı olarak yanıt vermesi talimatını verecektir.

'\nYou are a cockney fruit and vegetable seller.\nYour role is to assist your customer with their fruit and vegetable needs.\nRespond using cockney rhyming slang.\n\nTell me about the following fruit: {fruit}\n'

{} şablon içinde parantez kullanarak parametreler tanımlayabilirsiniz, örneğin {fruit}. Bu parametreler, prompt biçimlendirildiğinde değerlerle değiştirilecektir.

In [17]:
# prompt template'i kullanmak için programımızı değiştirelim:

from langchain.prompts import PromptTemplate

template = PromptTemplate(template="""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""", input_variables=["fruit"])

In [ ]:
# Biçimlendirilmiş prompt template'i input olarak geçirerek LLM'yi çağırın(call):
response = llm.invoke(template.format(fruit="apple")) #format: Parametreleri promptta geçirmek için

print(response)

Parametreleri prompt'a iletmek için (fruit = "elma") format yöntemini kullanırsınız. Input variables, prompt biçimlendirildiğinde doğrulanacak ve girişte herhangi bir değişken eksikse bir KeyError verceektir.


Prompt aşağıdaki gibi biçimlendirilecektir:


In [12]:
"""You are a cockney fruit and vegetable seller.

Your role is to assist your customer with their fruit and vegetable needs

Respond using cockney rhyming slang.

Tell me about the following fruit: apple"""

'You are a cockney fruit and vegetable seller.\n\nYour role is to assist your customer with their fruit and vegetable needs\n\nRespond using cockney rhyming slang.\n\nTell me about the following fruit: apple'

Programı çalıştırırken, aşağıdakilere benzer bir yanıt görmelisiniz:

In [13]:
"""Well, apples is a right corker - they come in all shapes and sizes from Granny Smiths"""

'Well, apples is a right corker - they come in all shapes and sizes from Granny Smiths'

Programı defalarca çalıştırırsanız, LLM her seferinde cevabı ürettiği için farklı yanıtlar aldığınızı fark edeceksiniz.

PromptTemplates Oluşturma

PromptTemplate.from_template() static method çağırarak bir stringden bir prompt oluşturabilir veya PromptTemplate.from_file() static method kullanarak bir dosyadan bir prompt yükleyebilirsiniz.

##### Configuring the LLM (LLM'i Yapılandırma)


LLM'yi oluşturduğunuzda, temperature ve model gibi parametrelerle yapılandırabilirsiniz.

In [ ]:
llm = OpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-3.5-turbo-instruct",
    temperature=0
)

Tüm promptların bir temperature değeri vardır. 0.0 ile 1.0 arasındadır ve yanıtın rastgeleliğini veya yaratıcılığını etkiler.
Değer 0'a yakın oldukça daha net yanıtlar sağlarken 1'e yaklaştıkça daha yaratıcı yanıtlar üretir ancak bu durum halliculation'a sebep olabilir.

Prompt Templates, yeniden kullanılabilir talimatlar oluşturmanıza olanak tanır.

Prompt Templates, birden fazla parametreyi kabul edebilir

##### Chains

LCEL

LangChain Expression Language (LCEL) kullanarak bir chain oluşturabilirsiniz. LCEL, Langchain componentlerini birlikte chainlemenin bir yoludur.
Componentler "|" operatorünü kullanarak birbirine zincirlenir(chain olur)

In [ ]:
chain = prompt | llm

Daha önce, meyve hakkında bir yanıt oluşturmak için hızlı bir şablon ve bir LLM kullanan bir program oluşturmuştuk.

In [ ]:
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate#

llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

template = PromptTemplate(template="""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""", input_variables=["fruit"])

response = llm.invoke(template.format(fruit="apple"))

print(response)

Bu programı bir chain ile birleştirebilir ve bir reusable component(yeniden kullanılabilir component) oluşturabiliriz.

In [ ]:
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
    )

template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

llm_chain = template | llm

response = llm_chain.invoke({"fruit": "apple"})

print(response)

llm_chain'in template ve llm'i chainleyerek nasıl oluşturulduğuna dikkat edin.

In [ ]:
llm_chain = template | llm

Template parametrelerini bir dictionary olarak geçirerek(invoke) llm_chain'i çağırırız.

In [ ]:
response = llm_chain.invoke({"fruit": "apple"})

##### Output Parsers


Chain'den gelen output genellikle bir string'dir ve outputu ayrıştırmak için bir output parser belirtebilirsiniz.
Chain'e bir StrOutputParser eklemek string olmasını sağlayacaktır.

In [ ]:
from langchain.schema import StrOutputParser

llm_chain = template | llm | StrOutputParser()

LLM'e belirli bir output türünü döndürme talimatı vermek için promptu değiştirebilirsiniz. Örneğin, Output JSON belirterek JSON'u döndürebilirsiniz.

In [ ]:
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Output JSON as {{"description": "your response here"}}

Tell me about the following fruit: {fruit}
""")

 SimpleJsonOutputParser'ı output_parser olarak belirterek Langchain'in yanıtı JSON olarak ayrıştırmasını sağlayabilirsiniz:


In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

llm_chain = template | llm | SimpleJsonOutputParser()

##### Chains kullanmanın faydaları:

Modularity(Modülerlik): Langchain, dil modeli uygulamaları oluşturmak için kullanılabilecek birçok modül sağlar. Bu modüller basit uygulamalarda tek başına (stand-alones) kullanılabilir ve daha karmaşık kullanım durumları için birleştirilebilir.

Customizability(Özelleştirilebilirlik): Çoğu Langchain uygulaması, LLM ve/veya kullanılan promptu yapılandırmanıza izin verir, bu nedenle bundan nasıl yararlanacağını bilmek büyük bir kolaylaştırıcı olacaktır.

Ease of Use:(Kullanım kolaylığı): Componentler, Langchain frameworkünün geri kalanını kullanıp kullanmadığınıza bakılmaksızın kullanımı kolay olacak şekilde tasarlanmıştır.

Standard Interface(Standart Arabirim): LangChain, geliştiricilerin tek bir LLM çağrısının ötesine geçen çağrı dizileri oluşturmasını sağlayan zincirler için standart bir arayüz sağlar.



Chains, modüler ve yeniden kullanılabilir bileşenler oluşturmak için kullanılır.
